Seleccionamos el piso del cual queremos mostrar la información en pantalla, y devolverá un fichero con el restaurante de cada cadena más cercano.

In [1]:
# Carga de las librerías, funciones, y ficheros necesarios.

import numpy as np 
import pandas as pd
import openpyxl

import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.model_selection import train_test_split

# Conecta con google drive
from google.colab import drive
drive.mount('/content/drive')

# Carga del dataset de pisos de airbnb Madrid filtrado (13258 reg) sólo por la ciudad Madrid.
airbnbMadrid = pd.read_csv('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/airbnblistingsMadridv1.csv', sep=';', decimal='.')

# Carga de los dataset de los restaurantes de las cadenas en Madrid.
restTacoBell = pd.read_excel('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/RestaurantesTacoBell.xlsx',)
restStarBucks = pd.read_excel('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/RestaurantesStarBucks.xlsx')
restKFC = pd.read_excel('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/RestaurantesKFC.xlsx')
restBurgerKing = pd.read_excel('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/RestaurantesBurgerKing.xlsx')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Se introduce por pantalla el numero de piso simulando que es la app de airbnb recopilando la informacion para mostrar al cliente

In [0]:
# Definimos una función para comprobar que el usuario introduce correctamente el ID del piso
def comprueba_int(): 
    while True: 
        try: 
            numEnt = int(input( "Introduzca ID del piso : ")) #Solicita ID piso
            return numEnt 
        except ValueError: 
            print('\nPor favor, introduzca un numero mayor que cero sin decimales\n')
            
# Definimos una función para comprobar que el número introducido sea mayor de 0 : 
def comprueba_mayor_cero(ID): 
    while True: 
        if ID > 0 : 
            return ID
        else:
            print("\nPor favor, introduzca un numero mayor que cero sin decimales\n") 
            ID = comprueba_int()

In [3]:
# Solicita el usuario introduzca el número de identificación del piso. Por ejemplo, el 15332216

IDPiso = 0        # Inicializa el campo a introducir, que deberá ser entero y mayor de cero

IDPiso = comprueba_mayor_cero(IDPiso)
print("\nEl ID introducido es :",IDPiso)

#IDPiso = 15332216

#airbnbMadridFastFood['ID'][IDPiso]  15332216


Por favor, introduzca un numero mayor que cero sin decimales

Introduzca ID del piso : 15332216

El ID introducido es : 15332216


In [0]:
import math 

# Función que calcula la distancia geográfica entre dos puntos en metros, pasándole la latitud y la longitud.
def distGeo(latitud1, longitud1, latitud2, longitud2): # posiciones de la vivienda, y los restaurantes
    rad=math.pi/180
    dlatitud=latitud2-latitud1
    dlongitud=longitud2-longitud1
    R=6372.795477598 # Radio de la Tierra
    # La fórmula tiene en cuenta la curvatura del planeta (aunque se podría calcular sin tenerla en cuenta, dado
    # que estamos en la misma ciudad).
    a=(math.sin(rad*dlatitud/2))**2 + math.cos(rad*latitud1)*math.cos(rad*latitud2)*(math.sin(rad*dlongitud/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

Busca el Taco Bell al piso seleccionado

In [0]:
# TacoBell
# Para el piso introducido, calculamos la distancia a todos los TacoBell de Madrid
distanciaMenor = 100  # EL restaurante a menor distancia. Se inicializa a un valor aleatorio. 
latPis = 0
latPiso = airbnbMadrid[airbnbMadrid['ID'] == IDPiso]['Latitude']  # latitud del piso 
lonPiso = airbnbMadrid[airbnbMadrid['ID'] == IDPiso]['Longitude'] # longitud del piso
# Para cada restaurante 
for j in range(len(restTacoBell)):
  # Calcula la distancia entre el IDpiso y todos los restaurantes
  distancia=abs(distGeo(latPiso,lonPiso,restTacoBell['Latit'][j],restTacoBell['Longit'][j]))
  #distancia=abs(distGeo(airbnbMadrid['Latitude'][i],airbnbMadrid['Longitude'][i], restTacoBell['Latitud'][j], restTacoBell['Longitud'][j]))
  if distancia < distanciaMenor: # si la última distancia calculada es menor que la distancia más pequeña tenemos
     distanciaMenor = distancia  # sustituir la distancia más pequeña que tenemos por la distancia calculada
     IDMenor = restTacoBell['ID'][j] # Almacenamos el ID del restaurante
     NombreMenor = restTacoBell['Lugar'][j]
     DireccionMenor = restTacoBell['Direcc'][j]
     LatitMenor = restTacoBell['Latit'][j]
     LongitMenor = restTacoBell['Longit'][j]                  

In [0]:
# Preparamos los datos del TacoBell
data=[[IDMenor],[NombreMenor],[DireccionMenor],[round(LatitMenor,6)],[round(LongitMenor,6)],[round(distanciaMenor,3)]]
datay=np.transpose(data)

In [0]:
# Creamos y rellenamos el dataframe de coordenadas para el Tableau con los datos del más cercano al piso
# ID : Codigo del establecimiento
# Nombre : nombre del establecimiento
# Direccion
# Latit y Longit : coordenas geográficas
# Distancia : distancia desde el piso seleccionado
dataID = pd.DataFrame(datay, columns=('ID','Nombre','Direccion','Latit','Longit','Distancia'))

Busca el StarBucks mas cercano al piso seleccionado

In [0]:
# StarBucks
# Para el piso introducido, calculamos la distancia a todos los StarBucks de Madrid
distanciaMenor = 100  # EL restaurante a menor distancia. Se inicializa a un valor aleatorio. 

# Para cada cafeteria 
for j in range(len(restStarBucks)):
  # Calcula la distancia entre el IDpiso y todos los StarBucks
  distancia=abs(distGeo(latPiso,lonPiso,restStarBucks['Latit'][j],restStarBucks['Longit'][j]))
  if distancia < distanciaMenor: # si la última distancia calculada es menor que la distancia más pequeña tenemos
     distanciaMenor = distancia  # sustituir la distancia más pequeña que tenemos por la distancia calculada
     IDMenor = restStarBucks['ID'][j] # Almacenamos el ID del restaurante
     NombreMenor = restStarBucks['Lugar'][j]
     DireccionMenor = restStarBucks['Direccion'][j]
     LatitMenor = restStarBucks['Latit'][j]
     LongitMenor = restStarBucks['Longit'][j]     

In [0]:
# Añadimos al dataset de salida los datos de StarBucks
data=[IDMenor,NombreMenor,DireccionMenor,round(LatitMenor,6),round(LongitMenor,6),round(distanciaMenor,3)]
datay=np.transpose(data)
dataID.loc[1] = datay

Busca el KFC mas cercano al piso seleccionado



In [0]:
# KFC
# Para el piso introducido, calculamos la distancia a todos los KFC de Madrid
distanciaMenor = 100  # EL restaurante a menor distancia. Se inicializa a un valor aleatorio. 

# Para cada KFC de Madrid : 
for j in range(len(restKFC)):
  # Calcula la distancia entre el IDpiso y todos los KFC
  distancia=abs(distGeo(latPiso,lonPiso,restKFC['Latit'][j],restKFC['Longit'][j]))
  if distancia < distanciaMenor: # si la última distancia calculada es menor que la distancia más pequeña tenemos
     distanciaMenor = distancia  # sustituir la distancia más pequeña que tenemos por la distancia calculada
     IDMenor = restKFC['ID'][j] # Almacenamos el ID del restaurante
     NombreMenor = restKFC['Lugar'][j]
     DireccionMenor = restKFC['Direccion'][j]
     LatitMenor = restKFC['Latit'][j]
     LongitMenor = restKFC['Longit'][j]     

In [0]:
# Añadimos al dataset de salida los datos de KFC
data=[IDMenor,NombreMenor,DireccionMenor,round(LatitMenor,6),round(LongitMenor,6),round(distanciaMenor,3)]
datay=np.transpose(data)
dataID.loc[2] = datay

Busca el Burger King mas cercano al piso seleccionado

In [0]:
# Burger King
# Para el piso introducido, calculamos la distancia a todos los BK de Madrid
distanciaMenor = 100  # EL restaurante a menor distancia. Se inicializa a un valor aleatorio. 

# Para cada BK de Madrid : 
for j in range(len(restBurgerKing)):
  # Calcula la distancia entre el IDpiso y todos los BK
  distancia=abs(distGeo(latPiso,lonPiso,restBurgerKing['Latit'][j],restBurgerKing['Longit'][j]))
  if distancia < distanciaMenor: # si la última distancia calculada es menor que la distancia más pequeña tenemos
     distanciaMenor = distancia  # sustituir la distancia más pequeña que tenemos por la distancia calculada
     IDMenor = restBurgerKing['ID'][j] # Almacenamos el ID del restaurante
     NombreMenor = restBurgerKing['Lugar'][j]
     DireccionMenor = restBurgerKing['Direccion'][j]
     LatitMenor = restBurgerKing['Latit'][j]
     LongitMenor = restBurgerKing['Longit'][j]     

In [0]:
# Añadimos al dataset de salida los datos de BK
data=[IDMenor,NombreMenor,DireccionMenor,round(LatitMenor,6),round(LongitMenor,6),round(distanciaMenor,3)]
datay=np.transpose(data)
dataID.loc[3] = datay

In [0]:
# Por ultimo, añadimos los datos del mismo piso

airbnbMadridID = airbnbMadrid.loc[airbnbMadrid.loc[:, 'ID'] == IDPiso]
airLatitPiso=airbnbMadridID['Latitude'].values
airLongitPiso=airbnbMadridID['Longitude'].values

data=[IDPiso,'Piso','Ud.esta aqui',round(airLatitPiso[0],6),round(airLongitPiso[0],6),0]
datay=np.transpose(data)
dataID.loc[4] = datay

In [15]:
dataID

,ID,Nombre,Direccion,Latit,Longit,Distancia
0,TB03,Taco Bell,calle montera 45,40.418596,-3.702306,1.816
1,SB03,StarBucks,Calle José Ortega y Gasset 29,40.430321,-3.682446,0.312
2,KFC11,KFC,Calle Goya 68,40.424798,-3.67692,0.839
3,BK16,BurgerKing,"Calle goya, 54",40.424874,-3.678224,0.742
4,15332216,Piso,Ud.esta aqui,40.428671,-3.685423,0


Saca por fichero los datos reunidos

In [0]:
# Guarda en fichero de salida los datos recopilados.
dataID.to_excel('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/airbnbMadridComidaRapida.xlsx')